### Preparing the data in HDF5 Formatting
Switching the .fits file to an HDF5 for better storage management and mainly practice

Importing the appropriate packages.

In [1]:
from astropy.table import Table
from astropy.io import fits
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
data = Table.read('/arc/home/aydanmckay/gaiahike/bp_rp_apogee.fits').to_pandas()
with pd.option_context('mode.use_inf_as_na', True):
    data.dropna(inplace=True)

In [3]:
data.shape

(481813, 230)

In [4]:
# idx = np.random.choice(int(len(d)*.9), size=len(d), replace=False)
# d_train = d[idx]
# d_test = d[~idx]
# len(d_test)
d_train, d_test = train_test_split(data,test_size=0.1)

In [5]:
len(data)

481813

In [6]:
len(d_train)

433631

In [7]:
# Example Python program that creates a hierarchy of groups
# and datasets in a HDF5 file using h5py
import h5py
import random
import numpy.random

# Create a HDF5 file
hierarchicalFileName  = "/arc/home/aydanmckay/mydata.h5"
with h5py.File(hierarchicalFileName, 'w') as hierarchicalFile:
    # Create a group under root
    train = hierarchicalFile.create_group("group_1")
    test = hierarchicalFile.create_group("group_2")

    train['data'] = np.array([d_train['fe_h'],d_train['logg'],d_train['teff']],#dtype=[('fe_h','f4'),('logg','f4'),('teff','f4')])
                            )
    train['label'] = np.array([d_train["bp_1"],d_train["bp_2"],d_train["bp_3"],d_train["bp_4"],d_train["bp_5"],d_train["bp_6"],d_train["bp_7"],
                               d_train["bp_8"],d_train["bp_9"],d_train["bp_10"],d_train["bp_11"],d_train["bp_12"],d_train["bp_13"],d_train["bp_14"],
                               d_train["bp_15"],d_train["bp_16"],d_train["bp_17"],d_train["bp_18"],d_train["bp_19"],d_train["bp_20"],d_train["bp_21"],
                               d_train["bp_22"],d_train["bp_23"],d_train["bp_24"],d_train["bp_25"],d_train["bp_26"],d_train["bp_27"],d_train["bp_28"],
                               d_train["bp_29"],d_train["bp_30"],d_train["bp_31"],d_train["bp_32"],d_train["bp_33"],d_train["bp_34"],d_train["bp_35"],
                               d_train["bp_36"],d_train["bp_37"],d_train["bp_38"],d_train["bp_39"],d_train["bp_40"],d_train["bp_41"],d_train["bp_42"],
                               d_train["bp_43"],d_train["bp_44"],d_train["bp_45"],d_train["bp_46"],d_train["bp_47"],d_train["bp_48"],d_train["bp_49"],
                               d_train["bp_50"],d_train["bp_51"],d_train["bp_52"],d_train["bp_53"],d_train["bp_54"],d_train["bp_55"],d_train["rp_1"],
                               d_train["rp_2"],d_train["rp_3"],d_train["rp_4"],d_train["rp_5"],d_train["rp_6"],d_train["rp_7"],d_train["rp_8"],
                               d_train["rp_9"],d_train["rp_10"],d_train["rp_11"],d_train["rp_12"],d_train["rp_13"],d_train["rp_14"],d_train["rp_15"],
                               d_train["rp_16"],d_train["rp_17"],d_train["rp_18"],d_train["rp_19"],d_train["rp_20"],d_train["rp_21"],d_train["rp_22"],
                               d_train["rp_23"],d_train["rp_24"],d_train["rp_25"],d_train["rp_26"],d_train["rp_27"],d_train["rp_28"],d_train["rp_29"],
                               d_train["rp_30"],d_train["rp_31"],d_train["rp_32"],d_train["rp_33"],d_train["rp_34"],d_train["rp_35"],d_train["rp_36"],
                               d_train["rp_37"],d_train["rp_38"],d_train["rp_39"],d_train["rp_40"],d_train["rp_41"],d_train["rp_42"],d_train["rp_43"],
                               d_train["rp_44"],d_train["rp_45"],d_train["rp_46"],d_train["rp_47"],d_train["rp_48"],d_train["rp_49"],d_train["rp_50"],
                               d_train["rp_51"],d_train["rp_52"],d_train["rp_53"],d_train["rp_54"],d_train["rp_55"]])
    # train['e_label'] = np.array([d_train["bpe_1"],d_train["bpe_2"],d_train["bpe_3"],d_train["bpe_4"],d_train["bpe_5"],d_train["bpe_6"],d_train["bpe_7"],
    #                            d_train["bpe_8"],d_train["bpe_9"],d_train["bpe_10"],d_train["bpe_11"],d_train["bpe_12"],d_train["bpe_13"],d_train["bpe_14"],
    #                            d_train["bpe_15"],d_train["bpe_16"],d_train["bpe_17"],d_train["bpe_18"],d_train["bpe_19"],d_train["bpe_20"],d_train["bpe_21"],
    #                            d_train["bpe_22"],d_train["bpe_23"],d_train["bpe_24"],d_train["bpe_25"],d_train["bpe_26"],d_train["bpe_27"],d_train["bpe_28"],
    #                            d_train["bpe_29"],d_train["bpe_30"],d_train["bpe_31"],d_train["bpe_32"],d_train["bpe_33"],d_train["bpe_34"],d_train["bpe_35"],
    #                            d_train["bpe_36"],d_train["bpe_37"],d_train["bpe_38"],d_train["bpe_39"],d_train["bpe_40"],d_train["bpe_41"],d_train["bpe_42"],
    #                            d_train["bpe_43"],d_train["bpe_44"],d_train["bpe_45"],d_train["bpe_46"],d_train["bpe_47"],d_train["bpe_48"],d_train["bpe_49"],
    #                            d_train["bpe_50"],d_train["bpe_51"],d_train["bpe_52"],d_train["bpe_53"],d_train["bpe_54"],d_train["bpe_55"],d_train["rpe_1"],
    #                            d_train["rpe_2"],d_train["rpe_3"],d_train["rpe_4"],d_train["rpe_5"],d_train["rpe_6"],d_train["rpe_7"],d_train["rpe_8"],
    #                            d_train["rpe_9"],d_train["rpe_10"],d_train["rpe_11"],d_train["rpe_12"],d_train["rpe_13"],d_train["rpe_14"],d_train["rpe_15"],
    #                            d_train["rpe_16"],d_train["rpe_17"],d_train["rpe_18"],d_train["rpe_19"],d_train["rpe_20"],d_train["rpe_21"],d_train["rpe_22"],
    #                            d_train["rpe_23"],d_train["rpe_24"],d_train["rpe_25"],d_train["rpe_26"],d_train["rpe_27"],d_train["rpe_28"],d_train["rpe_29"],
    #                            d_train["rpe_30"],d_train["rpe_31"],d_train["rpe_32"],d_train["rpe_33"],d_train["rpe_34"],d_train["rpe_35"],d_train["rpe_36"],
    #                            d_train["rpe_37"],d_train["rpe_38"],d_train["rpe_39"],d_train["rpe_40"],d_train["rpe_41"],d_train["rpe_42"],d_train["rpe_43"],
    #                            d_train["rpe_44"],d_train["rpe_45"],d_train["rpe_46"],d_train["rpe_47"],d_train["rpe_48"],d_train["rpe_49"],d_train["rpe_50"],
    #                            d_train["rpe_51"],d_train["rpe_52"],d_train["rpe_53"],d_train["rpe_54"],d_train["rpe_55"]])
    test['data'] = np.array([d_test['fe_h'],d_test['logg'],d_test['teff']],#dtype=[('fe_h','f4'),('logg','f4'),('teff','f4')])
                           )
    test['label'] = np.array([d_test["bp_1"],d_test["bp_2"],d_test["bp_3"],d_test["bp_4"],d_test["bp_5"],d_test["bp_6"],d_test["bp_7"],
                               d_test["bp_8"],d_test["bp_9"],d_test["bp_10"],d_test["bp_11"],d_test["bp_12"],d_test["bp_13"],d_test["bp_14"],
                               d_test["bp_15"],d_test["bp_16"],d_test["bp_17"],d_test["bp_18"],d_test["bp_19"],d_test["bp_20"],d_test["bp_21"],
                               d_test["bp_22"],d_test["bp_23"],d_test["bp_24"],d_test["bp_25"],d_test["bp_26"],d_test["bp_27"],d_test["bp_28"],
                               d_test["bp_29"],d_test["bp_30"],d_test["bp_31"],d_test["bp_32"],d_test["bp_33"],d_test["bp_34"],d_test["bp_35"],
                               d_test["bp_36"],d_test["bp_37"],d_test["bp_38"],d_test["bp_39"],d_test["bp_40"],d_test["bp_41"],d_test["bp_42"],
                               d_test["bp_43"],d_test["bp_44"],d_test["bp_45"],d_test["bp_46"],d_test["bp_47"],d_test["bp_48"],d_test["bp_49"],
                               d_test["bp_50"],d_test["bp_51"],d_test["bp_52"],d_test["bp_53"],d_test["bp_54"],d_test["bp_55"],d_test["rp_1"],
                               d_test["rp_2"],d_test["rp_3"],d_test["rp_4"],d_test["rp_5"],d_test["rp_6"],d_test["rp_7"],d_test["rp_8"],
                               d_test["rp_9"],d_test["rp_10"],d_test["rp_11"],d_test["rp_12"],d_test["rp_13"],d_test["rp_14"],d_test["rp_15"],
                               d_test["rp_16"],d_test["rp_17"],d_test["rp_18"],d_test["rp_19"],d_test["rp_20"],d_test["rp_21"],d_test["rp_22"],
                               d_test["rp_23"],d_test["rp_24"],d_test["rp_25"],d_test["rp_26"],d_test["rp_27"],d_test["rp_28"],d_test["rp_29"],
                               d_test["rp_30"],d_test["rp_31"],d_test["rp_32"],d_test["rp_33"],d_test["rp_34"],d_test["rp_35"],d_test["rp_36"],
                               d_test["rp_37"],d_test["rp_38"],d_test["rp_39"],d_test["rp_40"],d_test["rp_41"],d_test["rp_42"],d_test["rp_43"],
                               d_test["rp_44"],d_test["rp_45"],d_test["rp_46"],d_test["rp_47"],d_test["rp_48"],d_test["rp_49"],d_test["rp_50"],
                               d_test["rp_51"],d_test["rp_52"],d_test["rp_53"],d_test["rp_54"],d_test["rp_55"]])
    # test['e_label'] = np.array([d_test["bpe_1"],d_test["bpe_2"],d_test["bpe_3"],d_test["bpe_4"],d_test["bpe_5"],d_test["bpe_6"],d_test["bpe_7"],
    #                            d_test["bpe_8"],d_test["bpe_9"],d_test["bpe_10"],d_test["bpe_11"],d_test["bpe_12"],d_test["bpe_13"],d_test["bpe_14"],
    #                            d_test["bpe_15"],d_test["bpe_16"],d_test["bpe_17"],d_test["bpe_18"],d_test["bpe_19"],d_test["bpe_20"],d_test["bpe_21"],
    #                            d_test["bpe_22"],d_test["bpe_23"],d_test["bpe_24"],d_test["bpe_25"],d_test["bpe_26"],d_test["bpe_27"],d_test["bpe_28"],
    #                            d_test["bpe_29"],d_test["bpe_30"],d_test["bpe_31"],d_test["bpe_32"],d_test["bpe_33"],d_test["bpe_34"],d_test["bpe_35"],
    #                            d_test["bpe_36"],d_test["bpe_37"],d_test["bpe_38"],d_test["bpe_39"],d_test["bpe_40"],d_test["bpe_41"],d_test["bpe_42"],
    #                            d_test["bpe_43"],d_test["bpe_44"],d_test["bpe_45"],d_test["bpe_46"],d_test["bpe_47"],d_test["bpe_48"],d_test["bpe_49"],
    #                            d_test["bpe_50"],d_test["bpe_51"],d_test["bpe_52"],d_test["bpe_53"],d_test["bpe_54"],d_test["bpe_55"],d_test["rpe_1"],
    #                            d_test["rpe_2"],d_test["rpe_3"],d_test["rpe_4"],d_test["rpe_5"],d_test["rpe_6"],d_test["rpe_7"],d_test["rpe_8"],
    #                            d_test["rpe_9"],d_test["rpe_10"],d_test["rpe_11"],d_test["rpe_12"],d_test["rpe_13"],d_test["rpe_14"],d_test["rpe_15"],
    #                            d_test["rpe_16"],d_test["rpe_17"],d_test["rpe_18"],d_test["rpe_19"],d_test["rpe_20"],d_test["rpe_21"],d_test["rpe_22"],
    #                            d_test["rpe_23"],d_test["rpe_24"],d_test["rpe_25"],d_test["rpe_26"],d_test["rpe_27"],d_test["rpe_28"],d_test["rpe_29"],
    #                            d_test["rpe_30"],d_test["rpe_31"],d_test["rpe_32"],d_test["rpe_33"],d_test["rpe_34"],d_test["rpe_35"],d_test["rpe_36"],
    #                            d_test["rpe_37"],d_test["rpe_38"],d_test["rpe_39"],d_test["rpe_40"],d_test["rpe_41"],d_test["rpe_42"],d_test["rpe_43"],
    #                            d_test["rpe_44"],d_test["rpe_45"],d_test["rpe_46"],d_test["rpe_47"],d_test["rpe_48"],d_test["rpe_49"],d_test["rpe_50"],
    #                            d_test["rpe_51"],d_test["rpe_52"],d_test["rpe_53"],d_test["rpe_54"],d_test["rpe_55"]])
    # Print the groups
    print(hierarchicalFile["/"])
    print(train)
    print(test)
    print(train['data'])
    print(test["label"])

<HDF5 group "/" (2 members)>
<HDF5 group "/group_1" (2 members)>
<HDF5 group "/group_2" (2 members)>
<HDF5 dataset "data": shape (3, 433631), type "<f4">
<HDF5 dataset "label": shape (110, 48182), type "<f8">


In [8]:
with h5py.File("/arc/home/aydanmckay/mydata.h5", 'r') as f:
    d = f['group_2']['data']
    dset = d[:]
    print(f['group_2']['data'].shape[1])

48182


In [9]:
dset.shape

(3, 48182)

In [10]:
dset.shape[1]+len(d_train) == len(data)

True

In [11]:
len(data)

481813

In [12]:
d

<Closed HDF5 dataset>